In [1]:
import os
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Input, Conv2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix, classification_report
import numpy as np

In [2]:
from __future__ import print_function

import numpy as np
import warnings
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input

# Preparing the training and testing data


In [3]:
import splitfolders

splitfolders.ratio("../Datasets/Original/", # The location of dataset
                   output="../Datasets/Datasets_Divided/", # The output location
                   seed=22, # The number of seed
                   ratio=(.8, .1, .1), # The ratio of splited dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )



### Real-time Data Augmentation through ImageDataGenerator()

In [4]:
BATCH_SIZE = 64

train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     preprocessing_function=preprocess_input)

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

""" # Data Augmentation """

In [5]:

from sklearn import *
from skimage import io
import numpy as np
import os
from PIL import Image
datagen = ImageDataGenerator(        
            rotation_range=45,
            width_shift_range=0.2,  
            height_shift_range=0.2,
            brightness_range=[0.1, 0.7],
            shear_range=0.2,        
            zoom_range=0.3,        
            horizontal_flip=True, 
            vertical_flip=True,
            fill_mode='nearest', cval=125)


In [6]:

base_directory='../Datasets/Datasets_Divided/train/'
base_directory_target='../Datasets/Augmented/train/'
disease=os.listdir(base_directory)

for y in range(4):
    images=os.listdir(base_directory+str(disease[y]))
    dataset=[]
    for i ,image_name in enumerate(images):
        if(image_name.split('.')[1]=='png'):
            image=io.imread(base_directory+str(disease[y])+'/'+image_name)
            image=Image.fromarray(image,'RGB')
            dataset.append(np.array(image))

    x=np.array(dataset)
    i = 1
    for batch in datagen.flow(x, batch_size=1,save_to_dir=base_directory_target+str(disease[y]),save_prefix='aug',save_format='png'):        
        i += 1
        if i > (4*(x.shape[0])):  
            break



In [7]:

base_directory='../Datasets/Datasets_Divided/val/'
base_directory_target='../Datasets/Augmented/val/'
disease=os.listdir(base_directory)

for y in range(4):
    images=os.listdir(base_directory+str(disease[y]))
    dataset=[]
    for i ,image_name in enumerate(images):
        if(image_name.split('.')[1]=='png'):
            image=io.imread(base_directory+str(disease[y])+'/'+image_name)
            image=Image.fromarray(image,'RGB')
            dataset.append(np.array(image))

    x=np.array(dataset)
    i = 1
    for batch in datagen.flow(x, batch_size=1,save_to_dir=base_directory_target+str(disease[y]),save_prefix='aug',save_format='png'):        
        i += 1
        if i > (4*(x.shape[0])):  
            break


In [8]:
train_data_dir="../Datasets/Augmented/train/"
test_data_dir="../Datasets/Datasets_Divided/test/"
val_data_dir="../Datasets/Augmented/val/"

class_names=sorted(os.listdir("../Datasets/Augmented/train/"), reverse=True)[:4]
class_names_test=sorted(os.listdir("../Datasets/Datasets_Divided/test/"), reverse=True)[:4]
class_names_val = sorted(os.listdir("../Datasets/Augmented/val/"), reverse=True)[:4]
print(class_names)
print(class_names_test)
print(class_names_val)

['Normal', 'Monkeypox', 'Measles', 'Chickenpox']
['Normal', 'Monkeypox', 'Measles', 'Chickenpox']
['Normal', 'Monkeypox', 'Measles', 'Chickenpox']


In [9]:
class_subset = sorted(os.listdir("../Datasets/Augmented/train/"), reverse=True)[:4] # Using only the first 10 classes

In [10]:
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size = BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(val_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(224, 224),
                                             class_mode='categorical',
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)



Found 5115 images belonging to 4 classes.
Found 600 images belonging to 4 classes.
Found 81 images belonging to 4 classes.


# Using Pre-trained Layers for Feature Extraction

In [11]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG19 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = tf.keras.applications.vgg19.VGG19(include_top=False,
                                                  weights='imagenet',
                                                  input_tensor=None,
                                                  input_shape=input_shape,
                                                  pooling=None)
    
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

## Training VGG-19


In [12]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.0001)
n_classes=4

n_steps = traingen.samples // BATCH_SIZE

n_val_steps = validgen.samples // BATCH_SIZE

n_epochs = 25

# First we'll train the model without Fine-tuning
vgg_19_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)
vgg_19_model.summary()

Metal device set to: Apple M1


2023-04-07 00:25:17.819318: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-07 00:25:17.820131: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
from livelossplot.inputs.keras import PlotLossesCallback

plot_loss_1 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='vgg_19.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

#EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


In [ ]:
%%time 

vgg_19_history = vgg_19_model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1, early_stop, plot_loss_1],
                            verbose=1)

Epoch 1/25


2023-04-07 00:25:19.561219: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-07 00:25:20.230275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 5/79 [>.............................] - ETA: 23:52 - loss: 3.9310 - accuracy: 0.3406

In [ ]:
vgg_19_model.load_weights('vgg_19.weights.best.hdf5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_19_preds = vgg_19_model.predict(testgen)
vgg_19_pred_classes = np.argmax(vgg_19_preds, axis=1)



In [ ]:
vgg_19_acc = accuracy_score(true_classes, vgg_19_pred_classes)
print("VGG19 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_19_acc * 100))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix, classification_report

def fbeta(y_true, y_pred, threshold_shift=0):
    beta_squared = 4

    y_pred = K.clip(y_pred, 0, 1)
 
    y_pred_bin = K.round(y_pred + threshold_shift)
 
    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))
 
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
 
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())


In [ ]:
f1 = f1_score(vgg_19_pred_classes, true_classes, average='weighted')

print(classification_report(vgg_19_pred_classes, true_classes, target_names=class_names))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix, classification_report

class_names = testgen.class_indices.keys()

def plot_heatmap(y_true, y_pred, class_names, ax, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(
        cm, 
        annot=True, 
        square=True, 
        xticklabels=class_names, 
        yticklabels=class_names,
        fmt='d', 
        cmap=plt.cm.Blues,
        cbar=False,
        ax=ax
    )
    ax.set_title(title, fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    ax.set_ylabel('True Label', fontsize=12)
    ax.set_xlabel('Predicted Label', fontsize=12)

fig, (ax1) = plt.subplots(1, figsize=(15, 7))
  
plot_heatmap(true_classes, vgg_19_pred_classes, class_names, ax1, title="VGG-19 (Pre trained Model)")    

fig.suptitle("Confusion Matrix Model Comparison", fontsize=24,y=0.99)
fig.tight_layout(pad=8)
fig.subplots_adjust(top=.9)
plt.show()

In [ ]:
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from keras.preprocessing import image
from keras.utils import img_to_array, load_img


model = Model(inputs=[vgg_19_model.input], outputs=vgg_19_model.layers[17].output)
model.summary()


img_path = '../mpox.png'

img = load_img(img_path, target_size=(224, 224))

x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)



In [ ]:
# retrieve weights from block1_conv1
filters, biases = vgg_19_model.layers[1].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

plt.figure(figsize=(12,12))
n_filters, ix = 64, 1
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(3):
        # specify subplot and turn of axis
        ax = plt.subplot(14, 14, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        if j == 0:
            plt.imshow(f[:, :, j], cmap='Reds')
        elif j == 1:
            plt.imshow(f[:, :, j], cmap='Greens')
        elif j == 2:
            plt.imshow(f[:, :, j], cmap='Blues')
        ix += 1
        
# show the figure
plt.show()

In [ ]:
model = Model(inputs=[vgg_19_model.input], outputs=vgg_19_model.layers[17].output)
model.summary()
feature_maps = model.predict(x)
feature_maps.shape


In [ ]:
plt.figure(figsize=(12,12))
# plot the 64 maps in an 8x8 squares
square = 4
ix = 1
for _ in range(square):
    for _ in range(square):
        # specify subplot and turn of axis
        ax = plt.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gnuplot2') # 'RdBu','PRGn' 'CMRmap' 'gnuplot2'
        ix += 1
        
# show the figure
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
# plot the 64 maps in an 8x8 squares
square = 20
ix = 1
for _ in range(square):
    for _ in range(square):
        # specify subplot and turn of axis
        ax = plt.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='RdBu') # 'RdBu','PRGn' 'CMRmap' 'gnuplot2'
        ix += 1
        
# show the figure
plt.show()